In [2]:
import pandas as pd
import os

In [1]:
# execute this on command line - for some reason it doesn't work in notebook
!gsutil -m cp -R gs://urm-pr-experiment.appspot.com/experiment_results/* ./data

/bin/bash: line 1: gsutil: command not found


In [5]:
data_dir = 'data'
files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.csv')]
files

['data/f1f5e702-0d66-4d30-9e06-a49bd05ae6d8_digit_span_recall_1672759883091.csv']

In [6]:
pd.read_csv(files[0])

,rt,recall,stimuli,accuracy,trial_type,trial_index,time_elapsed,internal_node_id
0,2493,"4,1,6","4,1,6",1,digit-span-recall,6,9556,0.0-3.0-1.0
